In [1]:
import keras_tuner
import keras
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import TensorBoard

In [2]:
# Define the desired number of samples for each class
class_samples = [20300, 30200]  # Adjust these numbers as needed

# Calculate weights based on the desired number of samples
class_weights = [num_samples / sum(class_samples) for num_samples in class_samples]


# Generate a synthetic dataset with different numbers of samples for each class
X, y = make_classification(
    n_samples=sum(class_samples),
    n_features=4,
    n_informative=4,
    n_redundant=0,
    n_classes=2,
    weights=class_weights,
    random_state=42,
)


# Create a Pandas DataFrame with the generated data
columns = ["Feature_1", "Feature_2", "Feature_3", "Feature_4"]
synthetic_df = pd.DataFrame(data=X, columns=columns)

for column in synthetic_df:
    std = np.std(synthetic_df[column])
    mean = np.mean(synthetic_df[column])
    synthetic_df[column] = synthetic_df[column] - mean
    synthetic_df[column] = synthetic_df[column] / std


synthetic_df["target"] = y

# Display the first few rows of the synthetic dataset
synthetic_array = synthetic_df.values
(train_data, test_data, train_target, test_target) = train_test_split(
    synthetic_array, y, test_size=0.2, random_state=42
)

In [3]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Dense(hp.Choice("units", [8, 16, 32]), activation="relu"))
    activation_choice = hp.Choice("activation", ["sigmoid", "relu"])
    model.add(keras.layers.Dense(1, activation=activation_choice))
    model.compile(loss="mse")
    return model

In [4]:
tuner = keras_tuner.RandomSearch(build_model, objective="val_loss", max_trials=5)
# Define TensorBoard callback
tensorboard_callback = TensorBoard(log_dir="./logs")
tuner.search(
    train_data,
    train_target,
    epochs=5,
    validation_data=(test_data, test_target),
    callbacks=[tensorboard_callback],
)
best_model = tuner.get_best_models()[0]

In [6]:
best_model = tuner.get_best_models()[0]